In [4]:
import os
import pandas as pd
import sys
import re
import numpy as np

In [10]:
def summary(filePath, method, dataset):
    index = ['coverage_error', 'ranking_loss', 'hamming_loss', 'f1_macro',
       'f1_micro', 'Jaccard_Index', 'zero_one_error']
    df = pd.read_csv(filePath)
    mean = round(df.mean(axis=1),4)
    std = round(df.std(axis=1),4)
    df = pd.DataFrame(index=[index,[dataset]*7],columns=[method])
    df.loc[:,method] = [(x,y) for x,y in zip(mean,std)]
    return df

def get_result(filePath, method):
    result = pd.DataFrame()
    for f in os.listdir(filePath):
        if not f.startswith("."):
            f_path = os.path.join(filePath,f)
            dataset = re.search(method+"/(.*).csv$",f_path).group(1)
            summ = summary(f_path, method, dataset)
            result = pd.concat([result,summ],axis=0)    
    return result

In [11]:
def cal_mean(col):
    if type(col[0])==tuple:
        mean = round(np.cumsum([x[0] for x in list(col)])[-1]/len(col),4)
        std = round(np.cumsum([x[1] for x in list(col)])[-1]/len(col),4)
        return (mean,std)
    else:
        return round(np.cumsum([x for x in list(col)])[-1]/len(col),4)

def merge_result(methods, al, resultPath="./result/"):
    df = pd.DataFrame()
    for m in methods:
        filePath = os.path.join(resultPath, al, m)
        r = get_result(filePath, m)
        df = pd.concat([df,r],axis=1)
    df = df.sort_index()
    a = df.loc[["Jaccard_Index","f1_macro","f1_micro"],:]
    b = a.rank(ascending=False, method='min',axis=1)
    c_1 = pd.DataFrame(b, columns=a.columns, index=a.index)
    a = df.loc[["coverage_error","hamming_loss","ranking_loss","zero_one_error"],:]
    b = a.rank(ascending=True, method='min',axis=1)
    c_2 = pd.DataFrame(b, columns=a.columns, index=a.index)
    c = pd.concat([c_1,c_2],axis=0)
    df_final = pd.concat([df, c],axis=1)

    for m in df_final.index.levels[0]:
        a = df_final.loc[m,:]
        df_final.loc[(m,"average"),:] = 1
        df_final.loc[(m,"average"),:] = a.apply(cal_mean, axis=0)

    df_final = df_final.sort_index()
    return df_final

In [12]:
methods = ["random_NB_BCC_bayesNet","most_edges_NB_BCC_bayesNet","best_prediction_NB_BCC_bayesNet"]
resultPath = "./result/"
al = "ordering"
df_final = merge_result(methods, al, resultPath)
savePath = '/Users/jiangjunhao/Desktop/results_algorithms/'+al+".csv"
df_final.to_csv(savePath)

In [13]:
methods = ["tree_NB_BCC","bayesNet_NB_BCC"]
resultPath = "./result/"
al = "structure"
df_final = merge_result(methods, al, resultPath)
savePath = '/Users/jiangjunhao/Desktop/results_algorithms/'+al+".csv"
df_final.to_csv(savePath)

In [29]:
df = pd.DataFrame()

methods = ["CC_NB_bayesNet","BCC_NB_bayesNet","LEAD_NB_bayesNet"]
resultPath = "./result/"
al = "NB"
for m in methods:
    filePath = os.path.join(resultPath, al, m)
    r = get_result(filePath, m)
    df = pd.concat([df,r],axis=1)
    
methods = ["CC_SVM_bayesNet","BCC_SVM_bayesNet","LEAD_SVM_bayesNet"]
resultPath = "./result/"
al = "SVM"
for m in methods:
    filePath = os.path.join(resultPath, al, m)
    r = get_result(filePath, m)
    df = pd.concat([df,r],axis=1)
    
methods = ["CC_ESKDB_bayesNet","BCC_ESKDB_bayesNet","LEAD_ESKDB_bayesNet"]
resultPath = "./result/"
al = "ESKDB"
for m in methods:
    filePath = os.path.join(resultPath, al, m)
    r = get_result(filePath, m)
    df = pd.concat([df,r],axis=1)
    
df = df.sort_index()
a = df.loc[["Jaccard_Index","f1_macro","f1_micro"],:]
b = a.rank(ascending=False, method='min',axis=1)
c_1 = pd.DataFrame(b, columns=a.columns, index=a.index)
a = df.loc[["coverage_error","hamming_loss","ranking_loss","zero_one_error"],:]
b = a.rank(ascending=True, method='min',axis=1)
c_2 = pd.DataFrame(b, columns=a.columns, index=a.index)
c = pd.concat([c_1,c_2],axis=0)
df_final = pd.concat([df, c],axis=1)

for m in df_final.index.levels[0]:
    a = df_final.loc[m,:]
    df_final.loc[(m,"average"),:] = 1
    df_final.loc[(m,"average"),:] = a.apply(cal_mean, axis=0)

df_final = df_final.sort_index()
savePath = '/Users/jiangjunhao/Desktop/results_algorithms/'+"base_classifier"+".csv"
df_final.to_csv(savePath)

In [30]:
df_final

CC_NB_bayesNet   BCC_NB_bayesNet  LEAD_NB_bayesNet  \
Jaccard_Index  average   (0.7895, 0.0102)  (0.7979, 0.0051)  (0.7936, 0.0075)   
               emotions  (0.7533, 0.0099)  (0.7662, 0.0068)  (0.7631, 0.0124)   
               scene     (0.8593, 0.0066)  (0.8552, 0.0058)  (0.8446, 0.0074)   
               yeast     (0.7559, 0.0141)  (0.7724, 0.0027)   (0.773, 0.0027)   
coverage_error average   (4.2174, 0.1035)  (4.1854, 0.0895)  (4.3193, 0.4556)   
               emotions  (3.0394, 0.0528)   (3.0743, 0.075)  (3.0713, 0.0363)   
               scene     (1.6683, 0.0379)  (1.6717, 0.0272)   (2.0614, 1.168)   
               yeast     (7.9444, 0.2198)  (7.8103, 0.1664)  (7.8251, 0.1624)   
f1_macro       average     (0.4705, 0.02)  (0.4454, 0.0198)  (0.4183, 0.0697)   
               emotions    (0.5361, 0.02)  (0.5268, 0.0171)   (0.5102, 0.027)   
               scene     (0.6299, 0.0136)  (0.6273, 0.0146)  (0.5635, 0.1599)   
               yeast     (0.2454, 0.0263)  (0.1822, 0.0277)  (0.1812, 0.0222)   
f1_micro       average   (0.5943, 0.0156)  (0.5731, 0.0173)  (0.5523, 0.0537)   
               emotions  (0.5876, 0.0116)  (0.5914, 0.0174)  (0.5771, 0.0221)   
               scene     (0.6183, 0.0143)  (0.6122, 0.0158)  (0.5635, 0.1242)   
               yeast     (0.5769, 0.0209)  (0.5156, 0.0186)  (0.5163, 0.0147)   
hamming_loss   average   (0.2105, 0.0102)  (0.2021, 0.0051)  (0.2064, 0.0075)   
               emotions  (0.2467, 0.0099)  (0.2338, 0.0068)  (0.2369, 0.0124)   
               scene     (0.1407, 0.0066)  (0.1448, 0.0058)  (0.1554, 0.0074)   
               yeast     (0.2441, 0.0141)  (0.2276, 0.0027)   (0.227, 0.0027)   
ranking_loss   average   (0.1831, 0.0084)  (0.1798, 0.0072)  (0.2051, 0.0824)   
               emotions  (0.2195, 0.0089)   (0.2193, 0.011)  (0.2173, 0.0099)   
               scene     (0.1172, 0.0073)  (0.1175, 0.0054)  (0.1937, 0.2319)   
               yeast     (0.2126, 0.0089)  (0.2026, 0.0052)  (0.2043, 0.0055)   
zero_one_error average   (0.7456, 0.0206)  (0.7772, 0.0236)  (0.8145, 0.0369)   
               emotions  (0.7682, 0.0184)  (0.7639, 0.0268)  (0.7787, 0.0215)   
               scene     (0.5676, 0.0312)  (0.6037, 0.0272)  (0.7028, 0.0708)   
               yeast      (0.901, 0.0123)  (0.9639, 0.0167)  (0.9619, 0.0184)   

                          CC_SVM_bayesNet  BCC_SVM_bayesNet LEAD_SVM_bayesNet  \
Jaccard_Index  average   (0.8211, 0.0075)  (0.8271, 0.0035)  (0.8224, 0.0108)   
               emotions  (0.7884, 0.0126)  (0.7947, 0.0049)   (0.795, 0.0082)   
               scene     (0.8879, 0.0058)  (0.8922, 0.0032)  (0.8807, 0.0175)   
               yeast     (0.7869, 0.0041)  (0.7944, 0.0023)  (0.7915, 0.0066)   
coverage_error average    (4.248, 0.0861)   (4.2199, 0.063)  (4.4442, 0.4611)   
               emotions  (2.7997, 0.0507)  (2.7963, 0.0407)  (2.8208, 0.0544)   
               scene      (1.626, 0.0314)   (1.6036, 0.027)   (2.154, 1.1643)   
               yeast     (8.3183, 0.1763)  (8.2597, 0.1213)  (8.3577, 0.1647)   
f1_macro       average   (0.5601, 0.0222)  (0.5599, 0.0145)  (0.5253, 0.0713)   
               emotions  (0.6442, 0.0363)  (0.6369, 0.0182)  (0.6371, 0.0268)   
               scene     (0.6938, 0.0156)  (0.7007, 0.0096)   (0.591, 0.1741)   
               yeast     (0.3423, 0.0148)  (0.3421, 0.0158)  (0.3478, 0.0129)   
f1_micro       average   (0.6617, 0.0149)  (0.6617, 0.0087)  (0.6327, 0.0565)   
               emotions  (0.6638, 0.0221)  (0.6611, 0.0084)  (0.6604, 0.0135)   
               scene     (0.6851, 0.0159)  (0.6912, 0.0091)  (0.6069, 0.1438)   
               yeast     (0.6361, 0.0066)  (0.6329, 0.0086)  (0.6308, 0.0121)   
hamming_loss   average   (0.1789, 0.0075)  (0.1729, 0.0035)  (0.1776, 0.0108)   
               emotions  (0.2116, 0.0126)  (0.2053, 0.0049)   (0.205, 0.0082)   
               scene     (0.1121, 0.0058)  (0.1078, 0.0032)  (0.1193, 0.0175)   
               yeast     (0.2131, 0.0041)  (0.2056, 0.002